In [42]:
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
from datetime import datetime, timedelta


In [3]:
def convertTime(time):
    # covert time string to datetime object
    time = datetime.strptime(time, "%Y-%m-%d %H:%M:%S")
    # convert datetime object to timestamp
    timeStamp = int(datetime.timestamp(time))
    return timeStamp

In [4]:
def priceHistory(ticker, window, start, end):
    request = requests.session()
    #GET /markets/{market_name}/candles?resolution={resolution}&limit={limit}&start_time={start_time}&end_time={end_time}
    request = request.get('https://ftx.com/api/markets/' + str(ticker) 
                          + '/candles?resolution=' + str(window) 
                          + '&limit=5000'
                          + '&start_time='+ str(start)
                          + '&end_time=' + str(end)).json()
    request = request['result']
    price_history = pd.DataFrame(data=request)
    price_history['startTime'] = pd.to_datetime(price_history['startTime'])+timedelta(hours=10)
    price_history = price_history.drop('time', axis=1)
    return price_history

In [50]:
#data for requested ticker within a time interval
#window length in seconds. options: 
#15 = 15s
#60 = 1min 
#300 = 5min 
#900 = 15min
#3600 = 1h
#14400 = 4h
#86400 = 1d
window = 3600
ticker = 'ETH-PERP'
inputStart = '2020-09-30 00:00:00'
inputEnd = '2021-01-13 01:00:00' #set one period after the end time you want

def rolling_interval(inputStart, inputEnd,  ticker, interval):
    (outputStart, outputEnd)= convertTime(inputStart), convertTime(inputEnd)
    table = pd.DataFrame(priceHistory(ticker, interval, outputStart, outputEnd))
    while str(table.iloc[0,4])[:-6] != inputStart:
        newData = priceHistory(ticker, interval, convertTime(inputStart), convertTime(str(table.iloc[0,4])[:-6]))
        table = pd.concat([newData, table])
    # rearrange startTime to starting as 1st column
    table = table[['startTime','close', 'high', 'low', 'open', 'volume']]
    return table

data = rolling_interval(inputStart, inputEnd, ticker, window)
data['log return'] = np.log(data['close']).diff() #log returns
data

,startTime,close,high,low,open,volume,log return
0,2020-09-30 00:00:00+00:00,356.94,359.50,355.55,357.20,3.839720e+06,NaN
1,2020-09-30 01:00:00+00:00,354.99,358.00,354.78,356.94,4.007497e+06,-0.005478
2,2020-09-30 02:00:00+00:00,354.58,355.40,352.35,354.98,1.221408e+07,-0.001156
3,2020-09-30 03:00:00+00:00,353.66,354.71,351.57,354.58,6.474342e+06,-0.002598
4,2020-09-30 04:00:00+00:00,355.21,356.13,353.32,353.66,4.710694e+06,0.004373
...,...,...,...,...,...,...,...
2516,2021-01-12 20:00:00+00:00,1086.20,1124.30,1068.90,1117.20,7.625198e+07,-0.028140
2517,2021-01-12 21:00:00+00:00,1100.80,1108.20,1085.70,1086.20,3.799892e+07,0.013352
2518,2021-01-12 22:00:00+00:00,1088.30,1113.30,1084.30,1100.80,2.703046e+07,-0.011420
2519,2021-01-12 23:00:00+00:00,1025.90,1098.60,1025.00,1088.30,9.069426e+07,-0.059047


In [49]:
#https://stackoverflow.com/questions/40256338/calculating-average-true-range-atr-on-ohlc-data-with-python
def wwma(values, n):
    """
     J. Welles Wilder's EMA 
    """
    return values.ewm(alpha=1/n, adjust=False).mean()

def atr(df, n=14):
    data = df.copy()
    high = data['high']
    low = data['low']
    close = data['close']
    tr0 = abs(high - low)
    tr1 = abs(high - close.shift())
    tr2 = abs(low - close.shift())
    data['tr'] = pd.DataFrame(['tr0', 'tr1', 'tr2']).max(axis=1)
    data['atr'] = wwma(data['tr'], n)
    return data

atr(data)




DataError: No numeric types to aggregate

In [ ]:
# convert to csv
data.to_csv("ETHPERP.csv",header=True, index=False)